In [6]:
import ms_entropy as me
import pandas as pd

from matchms.importing import load_from_msp
from utils import generate_compound_name_options

In [2]:
query = list(load_from_msp("../data/filtered/simulated_matchms_filter_1%I_all_peaks.msp"))
reference = list(load_from_msp("../data/experimental/RECETOX_GC-EI_MS_20201028.msp"))

2024-07-15 10:01:06,593:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-07-15 10:01:06,594:WARNING:matchms:add_retention:None can't be converted to float.
2024-07-15 10:01:06,595:WARNING:matchms:add_retention:None can't be converted to float.
2024-07-15 10:01:06,598:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-07-15 10:01:06,598:WARNING:matchms:add_retention:None can't be converted to float.
2024-07-15 10:01:06,599:WARNING:matchms:add_retention:None can't be converted to float.
2024-07-15 10:01:06,600:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-07-15 10:01:06,601:WARNING:matchms:add_retention:None can't be converted to float.
2024-07-15 10:01:06,601:WARNING:matchms:add_retention:None can't be converted to float.
2024-07-15 10:01:06,603:WARNING:matchms:add_precursor_mz:No precursor_mz found in metadata.
2024-07-15 10:01:06,603:WARNING:matchms:add_retention:None can't be converted to float.
2024-07-15 10:01

In [3]:
peaks_query = {}
for x in query:
    peaks_query[x.get("compound_name")] = x.peaks.to_numpy

In [4]:
peaks_references = {}
for x in reference:
    peaks_references[x.get("compound_name")] = x.peaks.to_numpy

In [16]:
results = {
    "query": list(),
    "reference": list(),
    "scores": list()
}

for query_key, query_value in peaks_query.items():
    ref_options = generate_compound_name_options(query_key)
    for opt in ref_options:
        ref_value = peaks_references.get(opt)

        if ref_value is not None:
            entropy = me.calculate_entropy_similarity(query_value, ref_value, ms2_tolerance_in_da=0.0035)
            results["query"].append(query_key)
            results["reference"].append(opt)
            results["scores"].append(entropy)

In [17]:
results_df = pd.DataFrame(results)
results_df.to_csv("../data/output_matching/entropy/entropy_tol_0.0035da.tsv", sep="\t", index=False)